In [135]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [136]:
iOption0 = 0

In [137]:
isigma = 2
N_files = 1
N_centr = 5
N_hists = 2
N_layers = 6
N_ilyaers = 8
is_sigmalised = 0
iOption1 = 0
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [1,1,2,2,3,3,4,4,5,5,6,6]
if iOption0 == 1:
    N_layers = 6
    layer_bins = [1,1,2,2,3,3,4,4,5,5,6,6]
colors=[1,2,4,root.kGreen+2,root.kMagenta,3]
file_path="input/"
file_names="my-fifth_QA_19081_664runs"
hist_select_3D_names = ["d_dphi_hist","d_dthe_hist"]

In [138]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            #if icentr==4: hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{icentr-1}"))
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/my-fifth_QA_19081_664runs0.root


In [139]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.7,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [0.25, 0.35, 0.45, 0.6, 0.85, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.7, 1.0, 1.4, 2.0, 2.8, 4.9])


In [140]:
boards_tot = [[-0.04,0.04],[-0.04,0.04]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[0.01,0.03],[-0.03,-0.01]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]

In [141]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"/emc_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(seed0%2)][0], boards_fit[int(seed0%2)][1])
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[int(ilayer%2)][0], boards_fit2[int(ilayer%2)][1])#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(seed0%2)][0], boards_tot[int(seed0%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[int(seed0%2)][1])
        g_gaus2.SetParameter(1,0.)
        g_gaus2.SetParameter(2,0.003)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 1:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0%2)][0]/10,boards_tot[int(seed0%2)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0%2)][1]/10,boards_tot[int(seed0%2)][1]/2)
        total.SetParLimits(4,boards_fit2[int(ilayer%2)][0], boards_fit2[int(ilayer%2)][1])
        if ilayer>1:
            total.SetParLimits(1,boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
            total.SetParLimits(4,-100,100)

        total.SetParLimits(5,0.0005,0.005)

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0 ==0:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        if fits_mean==0:
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/d_delta_fits/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [142]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [143]:
import multiprocess as mp

Ntr = N_files

input_arr = [0,1]
if iOption0 == 1: input_arr = [0,1,2,3,4,5]
input_arr = [0,1,2,3,4,5]

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, input_arr, i, isigma) for i in range(Ntr)])
pool.close()

In [144]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [145]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2,root.kOrange]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.01512)
            fit_functions1[-1].SetParameter(1,0.00912)
            fit_functions1[-1].SetParLimits(1,0.001,0.018)
            fit_functions1[-1].SetParameter(2,-2.02)
            fit_functions1[-1].SetParLimits(2,-7.126,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [146]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.03
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [0.005260279485108861, 0.0036611367139432562, 0.002797349371783499, 0.002336467517705114, 0.00212213651459834, 0.0018649954927366895, 0.001683292192626908, 0.0015785065531392637, 0.017220824853372806]) array('d', [0.004356455439895254, 0.003348065272705473, 0.002720640537072105, 0.0023047585389650406, 0.002008309575442397, 0.0018114797685132974, 0.001633107907704671, 0.0015722173431002573, 0.0015421904222182052])


python ERROR: cannot open image file "output/Sigmas/sigma_000.png" for writing. Please check permissions.


In [147]:
print(f"params[{N_files}][{1}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
    print("},") 
    print("},")      
print("};")        

params[1][1][6][3]={
{
{
{0.001476, 0.006574, -3.832976},
{0.001462, 0.0057, -3.565458},
{0.002187, 0.00972, -2.629092},
{0.002065, 0.009191, -2.536502},
{0.002137, 0.010258, -1.955571},
{0.002307, 0.010977, -2.265782},
},
},
};


In [148]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][6][3]={
{
{
{0.001728, 0.011276, -4.915484},
{0.001753, 0.009893, -4.753083},
{0.001936, 0.010552, -2.424509},
{0.001901, 0.009681, -2.380318},
{0.002022, 0.008783, -1.692855},
{0.00196, 0.008976, -1.639149},
},
{
{0.001654, 0.008572, -4.220366},
{0.001612, 0.00773, -4.015908},
{0.001997, 0.010451, -2.509707},
{0.002002, 0.010409, -2.647375},
{0.001868, 0.005497, -1.10634},
{0.002142, 0.009606, -1.918805},
},
{
{0.001639, 0.007405, -3.974668},
{0.001615, 0.007128, -3.974891},
{0.002135, 0.010818, -2.761257},
{0.001997, 0.010113, -2.622972},
{0.002049, 0.009865, -1.884808},
{0.002053, 0.009862, -1.925203},
},
{
{0.001637, 0.006503, -3.736832},
{0.00161, 0.005628, -3.402454},
{0.002091, 0.009856, -2.538176},
{0.002126, 0.010334, -2.829712},
{0.002255, 0.010741, -2.153487},
{0.002222, 0.010669, -2.166249},
},
{
{0.001476, 0.006574, -3.832976},
{0.001462, 0.0057, -3.565458},
{0.002187, 0.00972, -2.629092},
{0.002065, 0.009191, -2.536502},
{0.002137, 0.010258, -1.955571},
{0.00

In [149]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            if iOption0 == 0:
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]/sqrt(x)+[2]/x",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.002*(0.5-ilayer))
                fit_functions1[-1].SetParameter(1,-0.004*(0.5-ilayer))
            if iOption0 == 1:
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [150]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.009, 0.009
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

python ERROR: cannot open image file "output/Sigmas/mean_000.png" for writing. Please check permissions.


In [151]:
print(f"params[{N_files}][{1}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_mean_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        if iOption0 ==0: print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        else: print("{"+f"{round(fits.GetParameter(0),6)}"+"},")
    print("},") 
    print("},")      
print("};")        

params[1][1][6][3]={
{
{
{0.000407, -0.000966, 0.001135},
{-0.000121, 0.000546, -0.000981},
{0.000731, -0.001828, 0.002659},
{-0.000274, 0.001333, -0.002484},
{0.000641, -0.001627, 0.00309},
{-0.000796, 0.002457, -0.003419},
},
},
};


In [152]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_mean_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][6][3]={
{
{
{0.001857, -0.003862, 0.002351},
{-0.001031, 0.002385, -0.001755},
{0.000797, -0.001978, 0.002516},
{-0.000281, 0.000876, -0.001942},
{0.000991, -0.002279, 0.003029},
{6.5e-05, 0.000286, -0.001916},
},
{
{0.000629, -0.001526, 0.001419},
{-0.000459, 0.001218, -0.001285},
{0.000188, -0.000866, 0.002221},
{-0.000126, 0.000564, -0.001996},
{0.000969, -0.002775, 0.00374},
{-8.5e-05, 0.000872, -0.002593},
},
{
{0.000467, -0.001105, 0.001261},
{-0.00018, 0.000593, -0.001059},
{0.000448, -0.001359, 0.002543},
{-0.000132, 0.000587, -0.002148},
{0.000184, -0.000904, 0.002859},
{-9e-05, 0.000892, -0.002778},
},
{
{0.00023, -0.000613, 0.001048},
{-6e-06, 0.000197, -0.000892},
{0.000245, -0.001032, 0.002451},
{-0.000164, 0.00086, -0.002336},
{0.00022, -0.000943, 0.002923},
{-0.000295, 0.001455, -0.003076},
},
{
{0.000407, -0.000966, 0.001135},
{-0.000121, 0.000546, -0.000981},
{0.000731, -0.001828, 0.002659},
{-0.000274, 0.001333, -0.002484},
{0.000641, -0.001627, 0.00309},